#### Description of the file

##### 0 - Setup

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import sklearn
import statsmodels as sm
import numba
from pathlib import Path

In [3]:
DIR_DATA = Path("..") / "data"
DIR_RESULTS = Path("..") / "results"
DIR_STOCK_DATA = DIR_DATA / "stock_data"

PATH_META = DIR_DATA / "metadata_listed.csv"
PATH_META_DELISTED = DIR_DATA / "metadata_delisted.csv"

In [4]:
df_meta = pd.read_csv(PATH_META)
df_target = df_meta.copy()

In [5]:
df_meta_delisted = pd.read_csv(PATH_META_DELISTED)
df_target_delisted = df_meta_delisted.copy()

In [6]:
all_stocks = []
p = 0.1

for row in tqdm.tqdm(df_target.itertuples(index=False), total=len(df_target), desc="Loading listed stocks"):
    x = np.random.binomial(1, p)
    if x == 0:
        continue
    
    symbol = row.Symbol
    path_stock = DIR_STOCK_DATA / f"{symbol}.csv"
    try:
        df_stock = pd.read_csv(path_stock)
    except Exception as e:
        continue

    df_stock.index = pd.MultiIndex.from_product([[symbol], df_stock.index], names=['ticker', 'date'])

    all_stocks.append(df_stock)

for row in tqdm.tqdm(df_target_delisted.itertuples(index=False), total=len(df_target_delisted), desc="Loading delisted stocks"):
    x = np.random.binomial(1, p)
    if x == 0:
        continue
    
    symbol = row.Symbol
    path_stock = DIR_STOCK_DATA / f"{symbol}.csv"
    try:
        df_stock = pd.read_csv(path_stock)
    except Exception as e:
        continue

    df_stock.index = pd.MultiIndex.from_product([[symbol], df_stock.index], names=['ticker', 'date'])

    all_stocks.append(df_stock)

df_stocks = pd.concat(all_stocks)

Loading listed stocks:   0%|          | 0/5088 [00:00<?, ?it/s]

Loading delisted stocks: 100%|██████████| 9720/9720 [00:17<00:00, 552.98it/s]


In [9]:
df_stocks.sample(10)

,,ticker,date,open,high,low,close,adj_close,volume
ticker,date,,,,,,,,
JSBA,459,JSBA,1995-02-01,16.500000,16.500000,16.500000,16.499964,16.499964,2600.0
CFS,2684,CFS,1982-09-03,4.250000,4.250000,4.250000,7.346729,7.346729,0.0
HHS,4770,HHS,2012-10-11,66.599998,66.900002,65.900002,54.334194,54.334194,7800.0
ODC,4127,ODC,1996-07-16,11.200000,11.200000,11.200000,5.074620,5.074620,21700.0
MGA,9397,MGA,2022-01-18,88.500000,89.089996,87.699997,77.028900,77.028900,1211600.0
PME,2142,PME,2020-02-06,1.030000,1.050000,1.020000,1.030000,1.030000,17200.0
KRUS,1447,KRUS,2025-05-05,60.980000,62.709999,60.529999,60.900002,60.900002,192400.0
CITP,141,CITP,2005-04-25,13.780000,13.780000,13.780000,13.779998,13.779998,3960.0
OZEMY,179,OZEMY,1997-02-11,11.250000,11.250000,11.250000,11.249928,11.249928,50100.0
